# Parsing Family Trees with Logic Programming

In [7]:
import json
from kanren import Relation, facts, run, conde, var, eq

## Queries

Note:

`conde((A, B, C), (D, E))` means `(A and B and C) or (D and E)`

Equivalent to the `(A, B, C); (D, E)` syntax in Prolog.

In [39]:
mother = Relation()
father = Relation()

def parent(x, y):
    # Is x the parent of y?
    return conde([father(x, y)], [mother(x, y)])
    # Is x the mother or father of y?

def grandparent(x, y):
    # Is x a grandparent of y?
    p = var()
    return conde((parent(x, p), parent(p, y)))
    # Is there a person p whose parent is x and who is a parent of y?

def sibling(x, y):
    # Is x a sibling of y?
    p = var()
    return conde((parent(p, x), parent(p, y)))
    # Is there a person p who is a parent of x and a parent of y?

def uncle(x, y):
    # Is x an uncle of y?
    p = var()
    return conde((father(p, x), grandparent(p, y)))
    # Is there a person p who is the father of x and the grandparent of y?

## Input Data and the Fact Base

`relationships.json` is a data file containing only parental relationships.  From this, we'll build a family tree.

In [40]:
with open("relationships.json", "r") as file:
    data = json.load(file)

for i in data['father']:
    # For each paternal relationship...
    dad = tuple(i)[0]
    kid = i[dad]
    
    facts(father, (dad, kid))

for i in data['mother']:
    # For each maternal relationship...
    mom = tuple(i)[0]
    kid = i[mom]
    
    facts(mother, (mom, kid))

## Making Queries

In [41]:
x = var()
m, d = var('mom'), var('dad')

# 0 means "all solutions"
# (m, d) means "result is a pair"

print("John's Children      :\t", run(0, x, father("John", x)))
print("William's Mother     :\t", run(0, x, mother(x, "William")))
print("Adam's Parents       :\t", run(0, x, parent(x, "Adam")))
print("Wayne's Grandparents :\t", run(0, x, grandparent(x, "Wayne")))
print("Megan's Grandchildren:\t", run(0, x, grandparent("Megan", x)))
print("David's Siblings     :\t", run(0, x, sibling("David", x)))
print("Tiffany's Uncles     :\t", run(0, x, uncle(x, "Tiffany")))
print("Spouses              :\t", run(0, (m, d), (father, d, c), (mother, m, c)))

John's Children      :	 ('David', 'Adam', 'William')
William's Mother     :	 ('Megan',)
Adam's Parents       :	 ('John', 'Megan')
Wayne's Grandparents :	 ('Megan', 'John')
Megan's Grandchildren:	 ('Julie', 'Sophia', 'Chris', 'Tiffany', 'Stephanie', 'Neil', 'Wayne', 'Peter')
David's Siblings     :	 ('Adam', 'David', 'William')
Tiffany's Uncles     :	 ('Adam', 'William', 'David')
Spouses              :	 (('Emma', 'William'), ('Olivia', 'David'), ('Megan', 'John'), ('Lily', 'Adam'))
